In [3]:
from PIL import Image
import pytesseract

In [6]:
print(pytesseract.image_to_string(Image.open('images/test.png')))


This is a lot of 12 point text to test the
ocr code and see if it works on all types
of file format.

The quick brown dog jumped over the
lazy fox. The quick brown dog jumped
over the lazy fox. The quick brown dog
jumped over the lazy fox. The quick
brown dog jumped over the lazy fox.



In [ ]:
import cohere
import numpy as np
import pandas as pd
import pdfplumber
import streamlit as st
from io import StringIO
from typing import Sequence
from dotenv import load_dotenv
import os
from numpy.linalg import norm

In [ ]:
load_dotenv()

co = cohere.Client(api)

In [ ]:
api=os.getenv('API_KEY')


In [ ]:
def extractTextFromPdf(pdf_path: str):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text()
    return text

def processTextInput(text: str, run_id: str = None):  
    text = StringIO(text).read()  
    CHUNK_SIZE=150
    chunks = [text[i:i + CHUNK_SIZE] for i in range(0, len(text), CHUNK_SIZE)]  

    df = pd.DataFrame.from_dict({'text': chunks}) 
    return df

def convertToList(df):
    df['col']=df[['text']].apply(lambda row: ' '.join(row.dropna().astype(str)), axis=1)
    seqOfStrings: Sequence[str]=df['col'].tolist()
    return seqOfStrings

def embed(Texts: Sequence[str]):
    res=co.embed(texts=Texts, model="small")
    return res.embeddings

In [ ]:
# options=st.selectbox("Input type", ["PDF","TEXT"])

# if options=="PDF":
#     pdf_file=st.file_uploader("Upload file", type=["pdf"])
#     if pdf_file is not None:
#         text=extractTextFromPdf(pdf_file)
#     if text is not None:
#         df=processTextInput(text)
# elif options == "TEXT":  
#     text = st.text_area("Paste the Document")  
#     if text is not None:  
#         df = processTextInput(text)

text=extractTextFromPdf('test/sample.pdf')

In [ ]:
df=processTextInput(text)
print(df)

In [ ]:
listOfText=convertToList(df)
embeddings=embed(listOfText)

In [ ]:
if df is not None:
    prompt="What is the main topic?"

In [ ]:
def topNNeighbours(prompt_embedding: np.ndarray, storage_embeddings: np.ndarray, df, n: int = 5):  
	if isinstance(storage_embeddings, list):  
		storage_embeddings = np.array(storage_embeddings)  
	if isinstance(prompt_embedding, list):  
		storage_embeddings = np.array(prompt_embedding)  
	similarity_matrix = prompt_embedding @ storage_embeddings.T / np.outer(norm(prompt_embedding, axis=-1), norm(storage_embeddings, axis=-1))  
	num_neighbors = min(similarity_matrix.shape[1], n)  
	indices = np.argsort(similarity_matrix, axis=-1)[:, -num_neighbors:]
	listOfStr=df.values.tolist()
	neighbourValues:list=[]
	for idx in indices[0]:
		neighbourValues.append(listOfStr[idx])
	# return len(listOfStr)
	# return indices
	return neighbourValues

In [ ]:
def generate(promptt):
    res=co.generate(prompt=promptt)
    return res

In [ ]:
if df is not None and prompt != "":
    base_prompt = "Based on the passage above, answer the following question:"
    prompt_embedding = embed([prompt])
    aug_prompts = topNNeighbours(np.array(prompt_embedding), embeddings, df)
    # print(aug_prompts)
    new_prompt = '\n'.join(str(idx) for idx in aug_prompts) + '\n\n' + base_prompt + '\n' + prompt + '\n'
    print(type(new_prompt))
    # is_success = False
    # while not is_success:
    #     try:
    #         response = generate(new_prompt)
    #         print(response)
    #         is_success = True
    #     except Exception:
    #         aug_prompts = aug_prompts[:-1]
    #         new_prompt = '\n'.join(aug_prompts) + '\n' + base_prompt + '\n' + prompt  + '\n'

    # st.write(response.generations[0].text)

In [ ]:
res=generate(new_prompt)

In [ ]:
print(res.generations[0].text)